# create df market

### loading libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

import time

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

# imports best practice pandas
import os

import numpy as np
import pandas as pd

#--------------------------------------------------------

### loading clean csv files

In [2]:
df_sales = pd.read_csv('./data/clean_data/cleaned_sales.csv')
df_prices = pd.read_csv('./data/clean_data/cleaned_prices.csv')

### exploring both df

In [3]:
df_sales.head(1)

,Unnamed: 0,id,store,date,sales
0,0,ACCESORIES_1_001_BOS_1,South_End,2011-01-31,0


In [4]:
df_prices.head(1)

,item,store_code,date,sell_price
0,ACCESORIES_1_001,BOS_1,2013-07-31,11.753875


In [5]:
del(df_sales["Unnamed: 0"])
del(df_prices["Unnamed: 0"])

KeyError: 'Unnamed: 0'

### create new column "id" so to merge by it

In [10]:
df_prices["id"] = df_prices["item"] + "_" + df_prices["store_code"]

### merge both df

In [11]:
df_market = pd.merge(df_sales,df_prices, on=["id", "date"], how="left")

In [12]:
df_market

,id,store,date,sales,item,store_code,sell_price
0,ACCESORIES_1_001_BOS_1,South_End,2011-01-31,0,NaN,NaN,NaN
1,ACCESORIES_1_001_BOS_1,South_End,2011-02-28,0,NaN,NaN,NaN
2,ACCESORIES_1_001_BOS_1,South_End,2011-03-31,0,NaN,NaN,NaN
3,ACCESORIES_1_001_BOS_1,South_End,2011-04-30,0,NaN,NaN,NaN
4,ACCESORIES_1_001_BOS_1,South_End,2011-05-31,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1951355,SUPERMARKET_3_827_PHI_3,Queen_Village,2015-12-31,41,SUPERMARKET_3_827,PHI_3,1.2
1951356,SUPERMARKET_3_827_PHI_3,Queen_Village,2016-01-31,49,SUPERMARKET_3_827,PHI_3,1.2
1951357,SUPERMARKET_3_827_PHI_3,Queen_Village,2016-02-29,46,SUPERMARKET_3_827,PHI_3,1.2
1951358,SUPERMARKET_3_827_PHI_3,Queen_Village,2016-03-31,68,SUPERMARKET_3_827,PHI_3,1.2


### drop rows with sell_price = null as we consider, items are without price because hadn´t arrive to store yet

In [13]:
df_market.dropna(subset=["sell_price"], inplace=True)

### make new column "income"

In [14]:
df_market["income"] =  df_market["sales"] * df_market["sell_price"]


In [15]:
df_market

,id,store,date,sales,item,store_code,sell_price,income
30,ACCESORIES_1_001_BOS_1,South_End,2013-07-31,10,ACCESORIES_1_001,BOS_1,11.753875,117.53875
31,ACCESORIES_1_001_BOS_1,South_End,2013-08-31,3,ACCESORIES_1_001,BOS_1,10.985800,32.95740
32,ACCESORIES_1_001_BOS_1,South_End,2013-09-30,4,ACCESORIES_1_001,BOS_1,10.985800,43.94320
33,ACCESORIES_1_001_BOS_1,South_End,2013-10-31,4,ACCESORIES_1_001,BOS_1,10.985800,43.94320
34,ACCESORIES_1_001_BOS_1,South_End,2013-11-30,10,ACCESORIES_1_001,BOS_1,10.985800,109.85800
...,...,...,...,...,...,...,...,...
1951355,SUPERMARKET_3_827_PHI_3,Queen_Village,2015-12-31,41,SUPERMARKET_3_827,PHI_3,1.200000,49.20000
1951356,SUPERMARKET_3_827_PHI_3,Queen_Village,2016-01-31,49,SUPERMARKET_3_827,PHI_3,1.200000,58.80000
1951357,SUPERMARKET_3_827_PHI_3,Queen_Village,2016-02-29,46,SUPERMARKET_3_827,PHI_3,1.200000,55.20000
1951358,SUPERMARKET_3_827_PHI_3,Queen_Village,2016-03-31,68,SUPERMARKET_3_827,PHI_3,1.200000,81.60000


### sort columns so to make df, easier to read

In [16]:
columns = ["id","item", "store_code","store", "date","sales","sell_price", "income"]

In [17]:
df_market = df_market[columns]

In [18]:
df_market

,id,item,store_code,store,date,sales,sell_price,income
30,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,South_End,2013-07-31,10,11.753875,117.53875
31,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,South_End,2013-08-31,3,10.985800,32.95740
32,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,South_End,2013-09-30,4,10.985800,43.94320
33,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,South_End,2013-10-31,4,10.985800,43.94320
34,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,South_End,2013-11-30,10,10.985800,109.85800
...,...,...,...,...,...,...,...,...
1951355,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,PHI_3,Queen_Village,2015-12-31,41,1.200000,49.20000
1951356,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,PHI_3,Queen_Village,2016-01-31,49,1.200000,58.80000
1951357,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,PHI_3,Queen_Village,2016-02-29,46,1.200000,55.20000
1951358,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,PHI_3,Queen_Village,2016-03-31,68,1.200000,81.60000


In [13]:
#file_name = "df_market_by_months.csv"
#df_market.to_csv(file_name)

### Group in months for work visualization